In [1]:
import random
import string
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import WordNetLemmatizer

warnings.filterwarnings('ignore')

In [2]:
# Downloading necessary NLTK data
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

True

In [6]:
# Reading in the corpus
with open('chatbot.txt', 'r', encoding='utf8', errors='ignore') as fin:
    raw = fin.read().lower()

In [7]:
# Tokenization
sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

# Preprocessing
lemmer = WordNetLemmatizer()

In [8]:
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [9]:
# Keyword Matching
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

In [10]:
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [11]:
# Generating response
def response(user_response):
    robo_response = ''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if req_tfidf == 0:
        robo_response = robo_response + "I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response + sent_tokens[idx]
        return robo_response

In [12]:
# Main function to handle conversation
def main():
    print("LEO: My name is Leobot. I will answer your queries about Chatbots. If you want to exit, type Bye!")
    flag = True
    while flag:
        user_response = input("You: ")
        user_response = user_response.lower()
        if user_response != 'bye':
            if user_response == 'thanks' or user_response == 'thank you':
                flag = False
                print("LEO: You are welcome..")
            else:
                if greeting(user_response) is not None:
                    print("LEO: " + greeting(user_response))
                else:
                    print("LEO: ", end="")
                    print(response(user_response))
                    sent_tokens.remove(user_response)
        else:
            flag = False
            print("LEO: Bye! take care..")

if __name__ == "__main__":
    main()

LEO: My name is Leobot. I will answer your queries about Chatbots. If you want to exit, type Bye!
You: hi
LEO: I am glad! You are talking to me
You: who coined chatbot?
LEO: the term "chatterbot" was originally coined by michael mauldin (creator of the first verbot, julia) in 1994 to describe these conversational programs.today, most chatbots are either accessed via virtual assistants such as google assistant and amazon alexa, via messaging apps such as facebook messenger or wechat, or via individual organizations' apps and websites.
You: who made first chatbot?
LEO: design
the chatbot design is the process that defines the interaction between the user and the chatbot.the chatbot designer will define the chatbot personality, the questions that will be asked to the users, and the overall interaction.it can be viewed as a subset of the conversational design.
You: Bye
LEO: Bye! take care..
